<a href="https://colab.research.google.com/github/FranciscoRMendes/graph_networks/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import trimesh

from SpectralGCN2 import SpectralGCN
from build_graph import create_adjacency_matrix
from heat import simulate_heat, visualize_heat, simulate_heat_with_even_sensors, visualize_heat_at_sensors, visualize_low_high_freq, visualize_heat_single_step_zoom
import random

In [ ]:
mesh = trimesh.creation.torus(major_radius=10, minor_radius=0.01)

In [2]:
import os

In [3]:
os.getcwd()

'/content'